In [ ]:
#The purpose of this notebook is to evaluate the embeddings on edge classification task.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import re
import networkx as nx
import math
from tqdm import tqdm
from gensim.models import Word2Vec
import warnings
from scipy import spatial

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix 
from sklearn.utils import shuffle

In [ ]:
#### Following Functions will extract important users mentioned or indicated to form nodes or vertices of Matrix

#user name are mentioned in URL of the columns "Twitter Reply to","Twitter Retweet of"
def getUserFromUrl(Urlstr):
    #print(Urlstr)
    tok = Urlstr.split('/')
    return tok[3]


#Declaring some global variables
All_User_for_Graph = set()   # Will consider all the users into account if mentioned tweeted retweeted for all the rows
All_OccuranceList = []
BigList = []
#-----

def GetTweetRetweetfromRow(row):
    All_User_for_Graph.add(row['AUTHOR_ID'])
    lis = []
    if(row["Thread Entry Type"] == 'post'):
        return lis
    else:
        ReTweetList = row["Twitter Retweet of"]
        if(len(ReTweetList) >0):
            for reTweet in ReTweetList:
                if(pd.isnull(reTweet)):
                    pass
                else:
                    name = getUserFromUrl(reTweet)
                    lis.append(name)
                    All_User_for_Graph.add(name)
        ReplyList = row['Twitter Reply to']
        if(len(ReplyList) >0):
            for reply in ReplyList:
                if(pd.isnull(reply)):
                    pass
                else:
                    name = getUserFromUrl(reply)
                    lis.append(name)
                    All_User_for_Graph.add(name)
        return list(set(lis))
                
def AllMentionedUser(row):
    All_User_for_Graph.add(row['AUTHOR_ID'])
    lis = []
    tweetList = row["Snippet"]
    for tweet in tweetList:
        lis.extend(re.findall(r"@(\w+)",str(tweet)))
    All_User_for_Graph.update(lis)
    return list(set(lis))

def AllConnections(row):
    conn_lis = []
    conn_lis.append(row['AUTHOR_ID'])
    conn_lis.extend(row['TweetReTweetUser'])
    conn_lis.extend(row['AllMentioneduser'])
    return list(set(conn_lis))

In [ ]:
def CreateTargetConnection(authorid):
    totalList= []
    for lis in BigList:
        if(lis.count(authorid) > 0):
            totalList.extend(lis)
    k = list(filter((authorid).__ne__, totalList))
    return k 

In [ ]:
def ReadMatfromPath(fileName):
    ReadFilepath = "C:/Users/SHAHBAZ/ZS UserEmbedding/BERT Embedding/" + str(fileName) + "_Matrix.txt"
    UserEmb = np.loadtxt(ReadFilepath)
    return UserEmb
def CombineMatrix(Mat1,Mat2):
    Mat = np.zeros((Mat1.shape[1] + Mat2.shape[1]))
    for i in range (1,Mat1.shape[0]):
        vec = []
        vec = np.concatenate((vec,Mat1[i]), axis=None)
        vec = np.concatenate((vec,Mat2[i]), axis=None)
        Mat = np.vstack((Mat,vec))
    return Mat   

In [ ]:
#### Function important for Deep Walk###
def get_randomwalk(node,path_length):
    random_walk = [node]
    for i in range(path_length-1):
        temp = list(G.neighbors(node))
        temp = list(set(temp) - set(random_walk))    
        if len(temp) == 0:
            break

        random_node = random.choice(temp)
        random_walk.append(random_node)
        node = random_node
        
    return random_walk

In [ ]:
#### Functions important for Node2Vec ####
def node2vec_walk(walk_length, start_node):
    walk = [start_node]
    while len(walk) < walk_length:
        cur = walk[-1]
        cur_nbrs = sorted(G.neighbors(cur))
        if len(cur_nbrs) > 0:
            if len(walk) == 1:
                walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
            else:
                prev = walk[-2]
                next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0],alias_edges[(prev, cur)][1])]
                walk.append(next)
        else:
            break

    return walk

def simulate_walks(num_walks, walk_length):
    walks = []
    nodes = list(G.nodes())
    print ('Walk iteration:')
    for walk_iter in range(num_walks):
        print (str(walk_iter+1), '/', str(num_walks))
        random.Random(7).shuffle(nodes)
        for node in nodes:
            walks.append(node2vec_walk(walk_length=walk_length, start_node=node))

    return walks

#give probabilty to edges on basis of p and q ( p is for dfs and q is for bfs)
def get_alias_edge(src, dst):
    unnormalized_probs = []
    for dst_nbr in sorted(G.neighbors(dst)):
        if dst_nbr == src:
            unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
        elif G.has_edge(dst_nbr, src):
            unnormalized_probs.append(G[dst][dst_nbr]['weight'])
        else:
            unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
    norm_const = sum(unnormalized_probs)
    normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

    return alias_setup(normalized_probs)


In [ ]:
def alias_setup(probs):
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)
    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()
        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)
    return J, q
#set values to nodes and edges
def preprocess_transition_probs():
    
    for node in G.nodes():
        unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
        norm_const = sum(unnormalized_probs)
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
        alias_nodes[node] = alias_setup(normalized_probs)

    for edge in G.edges():
        alias_edges[edge] = get_alias_edge(edge[0], edge[1])
        alias_edges[(edge[1], edge[0])] = get_alias_edge(edge[1], edge[0])
        
def alias_draw(J, q):
    K = len(J)
    kk = int(np.floor(np.random.rand()*K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [ ]:
############## Creating Training Data for Edge Classification #################
#Useful Methods
def non_neighbors_Custom(graph, node):
    nbors = list(set(graph.neighbors(node)))   #| set([node])
    lis = []
    for nnode in All_user_Node:
        if(nbors.count(nnode) == 0):
            lis.append(nnode)
    return lis

def getAllUserNodeNotNeighbours(Graph,node,numbers):
    non_neighbour = non_neighbors_Custom(Graph,node)
    random.Random(7).shuffle(non_neighbour)
    return non_neighbour[:numbers]

def GetJaccardCoef(G,usr1,usr2):
    cnbors = list(nx.common_neighbors(G, usr1, usr2))
    union_size = len(set(G[usr1]) | set(G[usr2]))
    score = len(cnbors) / union_size
    return score

def GetRessorceAllocationScore(G,usr1,usr2):
    score = sum(1 / G.degree(w) for w in nx.common_neighbors(G, usr1, usr2))
    return score

In [ ]:
def GetSimilarityScore(nodeA,nodeB,Matrix):
    rowd_usr1 = New_DF.loc[New_DF['AUTHOR_ID'] == nodeA]
    usr1_ind = rowd_usr1.index.values.astype(int)[0]
    rowd_usr2 = New_DF.loc[New_DF['AUTHOR_ID'] == nodeB]
    #print(nodeB)
    usr2_ind = rowd_usr2.index.values.astype(int)[0]
        
    usr1_vec = Matrix[usr1_ind]
    usr2_vec = Matrix[usr2_ind]
    res = 1 - spatial.distance.cosine(usr1_vec,usr2_vec)
    
    return abs(res)

def isUserTarget(cnnectionList):
    l = []
    for c in cnnectionList:
        if((All_user_Node.count(c)) > 0):
            l.append(c)
    return l

In [ ]:
def CalculateClassonCutoff(x1,x2,x3,cutoff):
    z =  classifier.intercept_[0] + classifier.coef_[0][0] * x1 + classifier.coef_[0][1] * x2 + classifier.coef_[0][2] * x3
    #z =  classifier.coef_[0][0] * x1 + classifier.coef_[0][1] * x2 + classifier.coef_[0][2] * x3
    pred_prob = 1/ (1 + np.exp(-z))
    if(pred_prob >= cutoff):
        return 1
    else:
        return 0

In [ ]:
def CalculateAccuracy_CutOff(xtest,ytest,cutoff):
    ypred = []
    for x in xtest:
        ypred.append(CalculateClassonCutoff(x[0],x[1],x[2],cutoff))
    cm = confusion_matrix(ytest, ypred)  
    
    precision = cm[0][0] / sum(cm[0])
    recall = cm[0][0] / (cm[0][0] + cm[1][0])
    accuracy = (cm[0][0] + cm[1][1]) / len(ypred)
    print(" Cut Off = " + str(cutoff) + "  Precision = " + str(precision) + " Recall = " + str(recall) + " Acurracy = " + str(accuracy))

In [ ]:
df = pd.read_excel(r'C:\Users\SHAHBAZ\Desktop\study mat\research papers\New folder\New folder\20200507_Cardiology_HCP_Data.xlsx')
Embedding_Column_Name = ["AUTHOR_ID", "AUTHOR_NAME","TRANS_AUTHOR_BIO","Account Type","Location Name","Gender","Mentioned Authors","Tags","Twitter Reply to","Twitter Retweet of","Thread Entry Type","Snippet"]
Main_DF = df[Embedding_Column_Name]
Main_DF = Main_DF.groupby('AUTHOR_ID',as_index= False)["AUTHOR_NAME","TRANS_AUTHOR_BIO","Account Type","Location Name","Gender","Mentioned Authors","Tags","Twitter Reply to","Twitter Retweet of","Thread Entry Type","Snippet"].agg(lambda x: list(x))

In [ ]:
#### Making a cleaner dataframe ####
for columName in Embedding_Column_Name[1:]:
    for _,x in Main_DF.iterrows():
        x[columName] = list(set(x[columName]))

In [ ]:
#Getting additional user information from the data:
Main_DF['TweetReTweetUser'] = [GetTweetRetweetfromRow(x) for _,x in Main_DF.iterrows()]
Main_DF['AllMentioneduser'] = [AllMentionedUser(x) for _,x in Main_DF.iterrows()]  
Main_DF['AllConnection'] = [AllConnections(x) for _,x in Main_DF.iterrows()] 
for _,x in Main_DF.iterrows():
    All_OccuranceList.extend(x['AllConnection'])
    BigList.append(x['AllConnection'])
All_User = list(All_User_for_Graph)

In [ ]:
Main_DF = Main_DF.head(4280)
Main_DF['Target_Connection'] = [CreateTargetConnection(x['AUTHOR_ID']) for _,x in Main_DF.iterrows()]

In [ ]:
All_user_Node = []
for _,x in Main_DF.iterrows():
    All_user_Node.append(x['AUTHOR_ID'])

In [ ]:
New_DF = Main_DF[["AUTHOR_ID","AllConnection","Target_Connection"]]

In [ ]:
### Creating graph and adding nodes###
G = nx.Graph()
for user in All_User:
    G.add_node(user)
    
### Connecting the edges for users who are linked ####
for _,x in New_DF.iterrows():
    source = x['AUTHOR_ID']
    lis = x['Target_Connection']
    
    for target in lis:
        G.add_edge(source, target)
        G[source][target]['weight'] = 1

In [ ]:
#### Implementing random walk ####
all_nodes = list(G.nodes())

random_walks = []
for n in tqdm(all_nodes):
    for i in range(10):
        random_walks.append(get_randomwalk(n,10))
warnings.filterwarnings('ignore')

In [ ]:
# train skip-gram (word2vec) model
model = Word2Vec(window = 4, sg = 1, size = 100, hs = 0,negative = 10,alpha=0.03, min_alpha=0.0007,seed = 14)
#size of 100,200 and 300 dimensions were tested and out of which 100 dimension performed best in MultiView
model.build_vocab(random_walks, progress_per=2)
model.train(random_walks, total_examples = model.corpus_count, epochs=20, report_delay=1)

In [ ]:
DeepWalkMat = np.zeros(100)
for _,x in New_DF.iterrows():
    DeepWalkMat = np.vstack((DeepWalkMat,model[x['AUTHOR_ID']]))
DeepWalkMat = DeepWalkMat[1:,:]

In [ ]:
######## Node2Vec #########
p = 1
q = 1 # p,q are the probabilties
alias_nodes = {}
alias_edges = {}
preprocess_transition_probs()
walks = simulate_walks(10,10)

In [ ]:
#skip gram model to generate embedding from walks
model = Word2Vec(window = 10, sg = 1, size = 100, hs = 0,negative = 10,alpha=0.03, min_alpha=0.0007,seed = 14)
#size of 100,200 and 300 dimensions were tested and out of which 100 dimension performed best in MultiView
model.build_vocab(walks, progress_per=2)
model.train(walks, total_examples = model.corpus_count, epochs=20, report_delay=1)

In [ ]:
Node2VecMat = np.zeros(100)
for _,x in New_DF.iterrows():
    Node2VecMat = np.vstack((Node2VecMat,model[x['AUTHOR_ID']]))
Node2VecMat = Node2VecMat[1:,:]

In [ ]:
#### Only considering nodes that are user and not a common topic of discussion
### This is necessary to fetch corresponding user embeddings
New_DF["UserTarget"] = [isUserTarget(x['Target_Connection']) for _,x in New_DF.iterrows()]

In [ ]:
####### Creating the DataFrame for Edge Classification without incorporating similarity score ######
count = 1
connections =[]

###defining ####
NodeA = []
NodeB = []
IsEdgeExist = []
Jaccard_Coeeficient_list = []
ResourceAllocationCoefflist = []

for _,x in New_DF.iterrows():
    node = x['AUTHOR_ID']
    conn = x['UserTarget']
    connections.clear()
    for c in conn:
        if((All_user_Node.count(c)) > 0):
            connections.append(c)
    non_lis = getAllUserNodeNotNeighbours(G,node,len(connections))
    for nnode in non_lis:
        NodeA.append(node)
        NodeB.append(nnode)
        Jaccard_Coeeficient_list.append(GetJaccardCoef(G,node,nnode))
        ResourceAllocationCoefflist.append(GetRessorceAllocationScore(G,node,nnode))
        IsEdgeExist.append(0)
    
    for conn in connections:
        NodeA.append(node)
        NodeB.append(conn)
        Jaccard_Coeeficient_list.append(GetJaccardCoef(G,node,conn))
        ResourceAllocationCoefflist.append(GetRessorceAllocationScore(G,node,conn))
        IsEdgeExist.append(1)
    
    if(count % 500 == 0):
        print("User " + str(count) + " completed out of " + str(len(All_user_Node)))
    count = count + 1

In [ ]:
data = {'main node': NodeA, 'neighbor node': NodeB, 'Resource Allocation Score': ResourceAllocationCoefflist, 'Jaccard coefficient score': Jaccard_Coeeficient_list,'Is_Edge_Exist':IsEdgeExist}
df = pd.DataFrame(data, columns = ['main node','neighbor node','Resource Allocation Score','Jaccard coefficient score','Is_Edge_Exist'])

In [ ]:
###Post Doc Basic Evaluation ####
### Read PostDoc matrix from path####
postDoc_600 = ReadMatfromPath('PostDoc_Basic')
postDoc_600 = postDoc_600[:4280,:]

In [ ]:
df.shape

In [ ]:
#### Train test split data after measuring the similarity score from the embedding matrix###
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],postDoc_600) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

In [ ]:
### Training the classifier to obtain coefficients corresponding to the predictor variables
X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
#### validating the data to obtain decision boundary
X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
### Testing the classifier on test data####
X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values
CalculateAccuracy_CutOff(X_test,Y_test,0.1)

In [ ]:
####Clinical BERT####
ClinicalBertSnippetEmbMat = ReadMatfromPath('ClinalBERT_SnippetEmbedding')
ClinicalBertAuthorInfoEmbMat = ReadMatfromPath('ClinalBERT_Autho_Info_Embedding')
ClinicalBertCombinedEmbMat = ReadMatfromPath('ClinalBERT_Combined_AutoEnc_Emb')

ClinicalBertCombinedEmbMat = ClinicalBertCombinedEmbMat[1:,:]
ClinicalBertSnippetEmbMat = ClinicalBertSnippetEmbMat[1:,:]
ClinicalBertAuthorInfoEmbMat = ClinicalBertAuthorInfoEmbMat[1:,:]

ClinicalBERT_1536_Mat = CombineMatrix(ClinicalBertSnippetEmbMat,ClinicalBertAuthorInfoEmbMat)

In [ ]:
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],ClinicalBERT_1536_Mat) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values

X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values

X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values

In [ ]:
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
CalculateAccuracy_CutOff(X_test,Y_test,0.25)

In [ ]:
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],ClinicalBertCombinedEmbMat) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values

X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values

X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values

In [ ]:
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
CalculateAccuracy_CutOff(X_test,Y_test,0.35)

In [ ]:
######Scibert ############
SciBertSnippetEmbMat = ReadMatfromPath('SciBERT_SnippetEmbedding')
SciBertAuthorInfoEmbMat = ReadMatfromPath('SciBERT_Autho_Info_Embedding')
SciBertCombinedEmbMat = ReadMatfromPath('SciBERT_Combined_Embedding')

SciBertSnippetEmbMat = SciBertSnippetEmbMat[1:,:]
SciBertAuthorInfoEmbMat = SciBertAuthorInfoEmbMat [1:,:]
SciBertCombinedEmbMat = SciBertCombinedEmbMat [1:,:]
SciBert_combine_1536 = CombineMatrix(SciBertSnippetEmbMat,SciBertAuthorInfoEmbMat)

In [ ]:
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],SciBert_combine_1536) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values

X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values

X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values

In [ ]:
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
CalculateAccuracy_CutOff(X_test,Y_test,0.1)

In [ ]:
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],SciBertCombinedEmbMat) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values

X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values

X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values

In [ ]:
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
CalculateAccuracy_CutOff(X_test,Y_test,0.1)

In [ ]:
#### Bio BERT #####
BioBertSnippetEmbMat = ReadMatfromPath('BioBERT_SnippetEmb')
BioBertAuthorInfoEmbMat = ReadMatfromPath('BioBERT_Autho_Info_Embedding')
BioBERT_1536_Mat = CombineMatrix(BioBertSnippetEmbMat,BioBertAuthorInfoEmbMat)
BioBERT_1536_Mat = BioBERT_1536_Mat[1: , :]
BioBert_reduced_300 = ReadMatfromPath('BioBERT_Combined_AutoEnc_Emb')
BioBert_reduced_300 = BioBert_reduced_300[1:,:]

In [ ]:
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],BioBERT_1536_Mat) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values

X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values

X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values

In [ ]:
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
CalculateAccuracy_CutOff(X_test,Y_test,0.1)

In [ ]:
df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],BioBert_reduced_300) for _,x in df.iterrows()]
train_df = df.head(20000)
test_df = df.tail(8912)
validation_df = test_df.head(1912)
test_df = test_df.tail(7000)

X = train_df.iloc[:, [2, 3,5]].values
Y = train_df.iloc[:, 4].values

X_val = validation_df.iloc[:, [2, 3,5]].values
Y_val = validation_df.iloc[:, 4].values

X_test = test_df.iloc[:, [2, 3,5]].values
Y_test = test_df.iloc[:, 4].values

In [ ]:
classifier = LogisticRegression(random_state = 0) 
classifier.fit(X, Y) 
print("Coefficients = ")
print(classifier.coef_)
print("Intercept") 
print(classifier.intercept_)

In [ ]:
c = 0.0 
while(c <= 1):
    CalculateAccuracy_CutOff(X_val,Y_val,c)
    c = c + 0.05

In [ ]:
CalculateAccuracy_CutOff(X_test,Y_test,0.15)

In [ ]:
### getting the matrix of embedding for multi view using Deep walk and Node to vec and combining with other embedding 
### from pretrained BERT models.
ClinicalBERT_1636_DW_Mat = CombineMatrix(ClinicalBERT_1536_Mat,DeepWalkMat)
ClinicalBERT_400_DW_Mat = CombineMatrix(ClinicalBertCombinedEmbMat,DeepWalkMat)
SciBERT_1636_DW_Mat = CombineMatrix(SciBert_combine_1536,DeepWalkMat)
SciBERT_400_DW_Mat = CombineMatrix(SciBertCombinedEmbMat,DeepWalkMat)
BioBERT_1636_DW_Mat = CombineMatrix(BioBERT_1536_Mat,DeepWalkMat)
BioBERT_400_DW_Mat = CombineMatrix(BioBert_reduced_300,DeepWalkMat)

ClinicalBERT_1636_Node2Vec_Mat = CombineMatrix(ClinicalBERT_1536_Mat,Node2VecMat)
ClinicalBERT_400_Node2Vec_Mat = CombineMatrix(ClinicalBertCombinedEmbMat,Node2VecMat)
SciBERT_1636_Node2Vec_Mat = CombineMatrix(SciBert_combine_1536,Node2VecMat)
SciBERT_400_Node2Vec_Mat = CombineMatrix(SciBertCombinedEmbMat,Node2VecMat)
BioBERT_1636_Node2Vec_Mat = CombineMatrix(BioBERT_1536_Mat,Node2VecMat)
BioBERT_400_Node2Vec_Mat = CombineMatrix(BioBert_reduced_300,Node2VecMat)

In [ ]:
### Defining a method that will evaluate all the predictions at a cutoff of 0.1 which was the trend.
def DoEvaluation(Matrix,name):
    df['SimilarityScore'] = [GetSimilarityScore(x['main node'],x['neighbor node'],Matrix) for _,x in df.iterrows()]
    train_df = df.head(20000)
    test_df = df.tail(8912)
    validation_df = test_df.head(1912)
    test_df = test_df.tail(7000)

    X = train_df.iloc[:, [2, 3,5]].values
    Y = train_df.iloc[:, 4].values

    X_val = validation_df.iloc[:, [2, 3,5]].values
    Y_val = validation_df.iloc[:, 4].values

    X_test = test_df.iloc[:, [2, 3,5]].values
    Y_test = test_df.iloc[:, 4].values
    
    print("################### "+ str(name) +" ###################################")
    
    classifier = LogisticRegression(random_state = 0) 
    classifier.fit(X, Y) 
    print("Coefficients = ")
    print(classifier.coef_)
    print("Intercept") 
    print(classifier.intercept_)
    
    CalculateAccuracy_CutOff(X_test,Y_test,0.1)

In [ ]:
####### Deep Walk #########
DoEvaluation(ClinicalBERT_1636_DW_Mat,'ClinicalBERT_1636_DW')
DoEvaluation(ClinicalBERT_400_DW_Mat,'ClinicalBERT_400_DW')
DoEvaluation(SciBERT_1636_DW_Mat,'SciBERT_1636_DW')
DoEvaluation(SciBERT_400_DW_Mat,'SciBERT_400_DW')
DoEvaluation(BioBERT_1636_DW_Mat,'BioBERT_1636_DW')
DoEvaluation(BioBERT_400_DW_Mat,'BioBERT_400_DW')

In [ ]:
###### Node2Vec #############
DoEvaluation(ClinicalBERT_1636_Node2Vec_Mat,'ClinicalBERT_1636_Node2Vec')
DoEvaluation(ClinicalBERT_400_Node2Vec_Mat,'ClinicalBERT_400_Node2Vec')
DoEvaluation(SciBERT_1636_Node2Vec_Mat,'SciBERT_1636_Node2Vec')
DoEvaluation(SciBERT_400_Node2Vec_Mat,'SciBERT_400_Node2Vec')
DoEvaluation(BioBERT_1636_Node2Vec_Mat,'BioBERT_1636_Node2Vec')
DoEvaluation(BioBERT_400_Node2Vec_Mat,'BioBERT_400_Node2Vec')